In [2]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt

import pandas as pd

import numpy as np
import math

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.cross_validation import cross_val_score

from collections import defaultdict

import csv

### Data Processing

#### Finance Data

In [14]:
zipToCounty = defaultdict(int)

with open('CSVs/NYzips.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        zipToCounty[row[0]] = row[1]

#### Clinton

In [15]:
df = pd.read_csv('CSVs/Clinton-NY.csv')
df2 = pd.DataFrame({'date' : df.contb_receipt_amt, 'amt' : df.contbr_occupation, 'zip9' : df.contbr_st})
df3 = df2.sort(['date'])
df3['cum_amt'] = df3.amt.cumsum()

In [16]:
df_len = len(df3.index)
bucket_size = int(df_len/1000)
idx_date = []
idx = 0
curr_date = df3.date[0]
for i in df3.date:
    if i != curr_date:
        curr_date = i
        idx += 1
    idx_date.append(idx)
df3['idx_date'] = idx_date

In [17]:
zip6 = []
county = []
for i in df3.zip9:
    if not math.isnan(i):
        z = str(int(i))
        if len(str(int(i))) > 5:
            z = z[:5]
        zip6.append(z)
        while not zipToCounty[z]:
            intZ = int(z)
            intZ -= 1
            z = str(intZ)
            if intZ < 10000:
                print i
                zipToCounty[z] = "None"
        county.append(zipToCounty[z])
    else:
        zip6.append("00000")
        county.append("None")
df3['zip6'] = zip6
df3['county'] = county

9309.0
9309.0
9309.0


In [18]:
clintonFinances = df3

#### Sanders

In [20]:
df = pd.read_csv('CSVs/Sanders-NY.csv')
df2 = pd.DataFrame({'date' : df.contb_receipt_amt, 'amt' : df.contbr_occupation, 'zip9' : df.contbr_st})
df3 = df2.sort(['date'])
df3['cum_amt'] = df3.amt.cumsum()

df_len = len(df3.index)
bucket_size = int(df_len/1000)
idx_date = []
idx = 0
curr_date = df3.date[0]
for i in df3.date:
    if i != curr_date:
        curr_date = i
        idx += 1
    idx_date.append(idx)
df3['idx_date'] = idx_date

zip6 = []
county = []
for i in df3.zip9:
    if not math.isnan(i):
        z = str(int(i))
        if len(str(int(i))) > 5:
            z = z[:5]
        zip6.append(z)
        while not zipToCounty[z]:
            intZ = int(z)
            intZ -= 1
            z = str(intZ)
            if intZ < 10000:
                print i
                zipToCounty[z] = "None"
        county.append(zipToCounty[z])
    else:
        zip6.append("00000")
        county.append("None")
df3['zip6'] = zip6
df3['county'] = county
sandersFinances = df3

10
10
10
10
10
10
10
1180
10
10
10
10
10
10
1180
1180
10
10
1180


/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:1159: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [21]:
# Features are [avg contribution, total raised, total contributions, {those 3 for the last 14 days}]
counties = [tup[0] for tup in list(clintonFinances.groupby('county'))]
numCounties = len(counties)
X_dems = [[] for i in range(numCounties)]

##### Sanders
clintonByCounty = clintonFinances.groupby('county')
countyAggregates = clintonByCounty['amt'].agg(['mean', 'sum', 'count'])

for i in range(numCounties):
    X_dems[i] += [countyAggregates['mean'][i], countyAggregates['sum'][i], countyAggregates['count'][i]]

clintonByDate = clintonFinances.groupby('idx_date')
startDate = 309
endDate = 323
for i in range(startDate, endDate):
    cbdByCounty = clintonByDate.get_group(i).groupby('county')
    timeAggregates = cbdByCounty['amt'].agg(['mean', 'sum', 'count'])
    for j in range(numCounties):
        if counties[j] in timeAggregates.index:
            X_dems[j] += [timeAggregates['mean'][counties[j]], timeAggregates['sum'][counties[j]], timeAggregates['count'][counties[j]]]
        else:
            X_dems[j] += [0, 0, 0]

##### Sanders
sandersByCounty = sandersFinances.groupby('county')
countyAggregates = sandersByCounty['amt'].agg(['mean', 'sum', 'count'])

for i in range(numCounties):
    X_dems[i] += [countyAggregates['mean'][i], countyAggregates['sum'][i], countyAggregates['count'][i]]

sandersByDate = sandersFinances.groupby('idx_date')
startDate = 290
endDate = 304
for i in range(startDate, endDate):
    cbdByCounty = sandersByDate.get_group(i).groupby('county')
    timeAggregates = cbdByCounty['amt'].agg(['mean', 'sum', 'count'])
    for j in range(numCounties):
        if counties[j] in timeAggregates.index:
            X_dems[j] += [timeAggregates['mean'][counties[j]], timeAggregates['sum'][counties[j]], timeAggregates['count'][counties[j]]]
        else:
            X_dems[j] += [0, 0, 0]
print len(X_dems)
X_dems.pop(counties.index("None"))
print len(X_dems)

63
62


In [22]:
compressed_idx = []
compressed_amt = []
compressed_cum_amt = []
i = 0
for index, row in df3.iterrows():
    if i % bucket_size == 0:
        compressed_idx.append(row['idx_date'])
        compressed_amt.append(row['amt'])
        compressed_cum_amt.append(row['cum_amt'])
    compressed_amt[-1] += row['amt']
    compressed_cum_amt[-1] += row['cum_amt']
    i += 1

compressed_df3 = pd.DataFrame({'date_idx' : compressed_idx, 'amt' : compressed_amt, 'cum_amt' : compressed_cum_amt})

In [25]:

with open('CSVs/NyPrimaryPollsR.csv', 'rb') as f:
    reader = csv.reader(f)
    for row in reader:
        print row

['Date', 'Sample', 'Margin oE', 'Trump', 'Kasich', 'Cruz']
['--', '--', '--', '60.4', '25.1', '14.5']
['4/6 - 4/17', '--', '--', '53.1', '22.8', '18.1']
['4/15 - 4/17', '361 LV', '5.1', '55', '21', '18']
['4/13 - 4/15', '705 LV', '5.9', '54', '19', '21']
['4/13 - 4/13', '481 LV', '4.5', '57', '22', '20']
['4/11 - 4/14', '14201 LV', '1', '49', '23', '14']
['4/10 - 4/13', '313 LV', '5.5', '54', '25', '16']
['4/7 - 4/10', '483 LV', '4.5', '51', '25', '20']
['4/6 - 4/11', '469 LV', '5', '50', '27', '17']
['4/6 - 4/11', '550 LV', '4.2', '55', '20', '19']
['4/6 - 4/10', '259 LV', '6.1', '54', '21', '18']
['4/5 - 4/10', '324 LV', '5.8', '60', '17', '14']
['4/6 - 4/7', '321 LV', '5.4', '56', '17', '22']
['4/6 - 4/7', '6041 LV', '3', '52', '23', '19']
['4/4 - 4/7', '602 LV', '4', '54', '22', '15']
['4/5 - 4/6', '857 LV', '3.3', '56', '24', '20']
['4/3 - 4/5', '302 LV', '5.6', '52', '25', '17']
['3/29 - 4/1', '657 LV', '5.6', '52', '20', '21']
['3/22 - 3/29', '457 LV', '4.6', '56', '19', '20']
[

In [27]:
x_dems = []
with open('CSVs/NyPrimaryPollsD.csv', 'rb') as f:
    reader = csv.reader(f)
#     for i, row in enumerate(reader):
#         if not i:
#             continue
#         print row

In [29]:
y_dems = [] # Hillary's percentages per county
with open('CSVs/NYPrimaryResultsRaw.csv', 'rb') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if not i:
            continue
        percent = float(row[1].replace(',', ''))/(int(row[1].replace(',', '')) + int(row[2].replace(',', '')))
        y_dems.append(percent)


### Random Forest

In [30]:
rng = np.random.RandomState(0)

# dataset = load_boston()
# X_full, y_full = dataset.data, dataset.target
X_full = np.array(X_dems)
y_full = np.array(y_dems)
n_samples = X_full.shape[0]
n_features = X_full.shape[1]

# Estimate the score on the entire dataset, with no missing values
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_full, y_full).mean()
print("Score with the entire dataset = %.2f" % score)

Score with the entire dataset = -0.72


In [32]:
estimator.fit(X_full, y_full)
summ = 0
print len(y_full)
for i in range(62):
    pred = estimator.predict(X_full[i])
    actual = y_full[i]
    predWin = "Lose"
    if pred > .5:
        predWin = "Win"
    actualWin = "Lose"
    if actual > .5:
        actualWin = "Win"
    print (predWin, actualWin)
    summ += predWin != actualWin
print summ, summ/62.

62
('Lose', 'Lose')
('Lose', 'Lose')
('Win', 'Win')
('Lose', 'Lose')
('Lose', 'Lose')
('Win', 'Lose')
('Win', 'Lose')
('Win', 'Lose')
('Lose', 'Lose')
('Lose', 'Lose')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
('Win', 'Win')
3 0.0483870967742
